# Import Library

In [51]:
import cv2 
from IPython.display import Video, HTML
import numpy as np

## Intial Video

In [45]:
input_video = 'motion_test.mp4'
Video(input_video)

In [46]:
video_cap = cv2.VideoCapture(input_video)
if not video_cap.isOpened():
    print('Unable to Open: ' + input_video)

frame_w = int(video_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_h = int(video_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(video_cap.get(cv2.CAP_PROP_FPS))

size = (frame_w, frame_h)
size_quad = (int(2 *frame_w), int(2*frame_h))

video_out_quad = cv2.VideoWriter('video_out_quad.mp4', cv2.VideoWriter_fourcc(*'XVID'), fps, size_quad)

In [47]:
def DrawBanner(frame, text, banner_height_percent=0.08, font_scale =0.8, text_color=(0,255, 0), font_thickness=2):
    banner_height = int(banner_height_percent * frame.shape[0])
    cv2.rectangle(frame, (0,0), (frame.shape[1], banner_height), (0,0,0), thickness=-1)
    left_offset =20
    location =(left_offset, int(10 + (banner_height_percent * frame.shape[0])/2))
    cv2.putText(frame, text, location, cv2.FONT_HERSHEY_SIMPLEX, font_scale, text_color, font_thickness, cv2.LINE_AA)

In [48]:
background_subtractor = cv2.createBackgroundSubtractorKNN(history=4, dist2Threshold=4, detectShadows=True)


ksize = (5,5)
red =(0, 0,255)
yellow = (0, 255, 255)

while True:
    ret, frame = video_cap.read()

    if frame is None:
        break
    else:
        frame_erode = frame.copy()
    
    fg_mask = background_subtractor.apply(frame)
    motion_area = cv2.findNonZero(fg_mask)
    x, y, w, h = cv2.boundingRect(motion_area)

    fg_mask_erode = cv2.erode(fg_mask, np.ones(ksize, np.uint8))
    motion_area_erode = cv2.findNonZero(fg_mask_erode)
    xe, ye, we, he = cv2.boundingRect(motion_area_erode)

    if motion_area is not None:
        cv2.rectangle(frame, (x,y), (x+w, y+h), red, thickness=6)
    
    if motion_area_erode is not None:
        cv2.rectangle(frame_erode, (xe, ye), (xe+we, ye+he), red, thickness=6)
    
    frame_fg_mask = cv2.cvtColor(fg_mask, cv2.COLOR_GRAY2BGR)
    frame_fg_mask_erode = cv2.cvtColor(fg_mask_erode, cv2.COLOR_GRAY2BGR)

    DrawBanner(frame_fg_mask, 'Foreground Mask')
    DrawBanner(frame_fg_mask_erode, 'Foreground Mask Eroded')

    frame_top = np.hstack([frame_fg_mask, frame])
    frame_bot = np.hstack([frame_fg_mask_erode, frame_erode])
    frame_composite = np.vstack([frame_top, frame_bot])

    fc_h, fc_w, _ = frame_composite.shape
    cv2.line(frame_composite, (0, int(fc_h/2)), (fc_w, int(fc_h/2)), yellow, thickness=1, lineType=cv2.LINE_AA)

    video_out_quad.write(frame_composite)

video_cap.release()
video_out_quad.release()

## Output Video

In [53]:
HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/fdMwtmIRiy4" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" allowfullscreen></iframe>')